# MCore Jupyter

Miking (Meta vIKING) is a meta language system for creating embedded domain-specific and general-purpose languages. Miking is not a programming language, but rather a language system for
creating languages and generating efficient compilers.

This notebook showcases the Miking Core (MCore) Jupyter kernel's capabilities. When going through the notebook, we recommend playing around with it by changing the values in code cells or trying out your own examples.

## Table of Contents
1. [Basic Examples](#MCore)
2. [Python Intrinsics](#Python-Intrinsics)
3. [Additional Notebook Features](#Notebook-Features)

## Basic Examples

The Jupyter kernel supports all features of the MCore language, excluding utests. The following cells show some examples.

In [ ]:
print "Hello\n"

In [ ]:
let x = addi 1 2 in
x

In [ ]:
let double = lam x. muli x 2
let foo = lam x. lam y. addi x y

In [ ]:
recursive
let odd = lam n.
    if eqi n 1 then true
    else if lti n 1 then false
    else even (subi n 1)
let even = lam n.
    if eqi n 0 then true
    else if lti n 0 then false
    else odd (subi n 1)
end

In [ ]:
(odd 4, even 4)

In [ ]:
let x_test = 5

In [ ]:
type Tree
con Node : (Tree,Tree) -> Tree
con Leaf : (Int) -> Tree

let tree = Node(Node(Leaf 4, Leaf 2), Leaf 3)

recursive
  let count = lam tree.
    match tree with Node t then
      let left = t.0 in
      let right = t.1 in
      addi (count left) (count right)
    else match tree with Leaf v then v
    else error "Unknown node"
end

In [ ]:
count tree

In [ ]:
include "string.mc"
include "seq.mc"

mexpr
int2string 23

In [ ]:
map double [1,2,3]

In [ ]:
lang Arith
  syn Expr =
  | Num Int
  | Add (Expr, Expr)

  sem eval =
  | Num n -> Num n
  | Add (e1,e2) ->
    match eval e1 with Num n1 then
      match eval e2 with Num n2 then
        Num (addi n1 n2)
      else error "Not a number"
    else error "Not a number"
end

mexpr
use Arith in
eval (Add (Num 2, Num 3))

In [ ]:
lang Bool
  syn Expr =
  | True()
  | False()
  | If (Expr, Expr, Expr)

  sem eval =
  | True() -> True()
  | False() -> False()
  | If(cnd,thn,els) ->
    let cndVal = eval cnd in
    match cndVal with True() then eval thn
    else match cndVal with False() then eval els
    else error "Not a boolean"
end

mexpr
use Bool in
eval (If (True(),False(),True()))

In [ ]:
lang ArithBool = Arith + Bool
  syn Expr =
  | IsZero Expr

  sem eval =
  | IsZero e ->
    match eval e with Num n then
      if eqi n 0 then True() else False()
    else
      error "Not a number"
end

In [ ]:
use ArithBool in
eval (If (IsZero (Num 0), (Num 1), (Num 3)))

## Python Intrinsics

An optional feature of MCore is Python intrinsics, which allow calling Python functions from MCore code. The Jupyter kernel includes these features.

The following example shows how to use the intrinsics to sort a sequence using
Python's builtin `sorted` function. Before you can call a Python function, you will need to import the relevant Python module using `pyimport`.

In [ ]:
let builtins = pyimport "builtins"

Any module in the Python path can be imported in this way. Now, `pycall` can be used to call a function from that module.

In [ ]:
let x = [5,4,2,1,3]
let y = pycall builtins "sorted" (x,)

In [ ]:
y

The result of the `pycall` is a Python value. Python values can be passed to other Python functions, but not regular MCore functions:

In [ ]:
pycall builtins "print" (y,)

In [ ]:
map (addi 2) y

To recover an MCore value from a Python value, use the `pyconvert` intrinsic.

In [ ]:
let y_mcore = pyconvert y in
map (addi 2) y_mcore

Most basic values can be converted between Python and MCore types; the main exceptions are Python classes and MCore user-defined data types. For a detailed explanation, see [the main README](./README.md).

## Additional Notebook Features

In addition to what we've already seen, the Jupyter kernel also offers some additional features.

### Autocompletion
One thing that you may not have noticed is that autocompletion is available. To get completions, use `Tab` after starting to type a name. Try it out in the cell below.

In [ ]:
option

### Python Cells

The MCore kernel also allows executing Python code and interacting with it from
MCore. Use the special directive `%%python` at the top of a cell to evaluate
Python code.

For example, the following cell defines a Python function `foo` and calls it.

In [ ]:
%%python
def foo(x):
  print("foo"+x)

foo("bar")

You can call the functions you have defined in Python cells in normal MCore
cells by using the Python intrinsics. A user-defined function can
be called by importing and using the Python module `__main__`. For example,
consider the following cell:

In [ ]:
let m = pyimport "__main__" in
let x = "baz" in
pycall m "foo" (x,)

This cell calls the Python function `foo` defined above, printing
`foobaz` as expected.

### Plotting Graphs
It is possible to plot graphs using the Python library `matplotlib`.
The Jupyter kernel offers integration with `matplotlib` to display plots
directly in a notebook.

To use this functionality, first make sure that `matplotlib` is installed (if
not, you can install it using `pip`). Now, when you use `matplotlib`'s plot
functions in a notebook cell, the plots will be displayed as part of the cell's
output. For example, try the following cell:

In [ ]:
let plt = pyimport "matplotlib.pyplot"
let x = [1,2,4,8]
let _ = pycall plt "plot" (x,)

While this example uses the Python intrinsics, running the plot code directly in
a Python cell would of course also work.

### IPM Visualization
The notebook also supports visualization using the IPM project's server. If the IPM visualization server is installed correctly, the following example should work:

In [ ]:
-- Update this with the path to the IPM repo
include "/path/to/ipm/src/models/modelVisualizer.mc"

In [ ]:
-- create your DFA
let states = ["s0","s1","s2","s3"]
let transitions = [
("s0","s1",'1'),
("s1","s1",'1'),
("s1","s2",'0'),
("s2","s1",'1'),
("s2","s3",'0'),
("s3","s1",'1')
]
let startState = "s0"
let acceptStates = ["s3"]
let char2string = (lam b. [b])
let dfa = dfaConstr states transitions startState acceptStates eqstr eqchar
let model = DFA(dfa,"1001010",identity,char2string,"RL",[("s0","start state"),("s3","accept state")])

After creating our model, the directive `%%visualize` can be used to produce the visualization. Note that the `formatModels` function outputs a string containing a JSON representation of the model to be visualized.

In [ ]:
%%visualize
formatModels [model]